In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Read the data
X = pd.read_csv('train.csv', index_col='Employee_ID')
X_test_full = pd.read_csv('test.csv', index_col='Employee_ID')


In [3]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['Attrition_rate'], inplace=True)
y = X.Attrition_rate          
X.drop(['Attrition_rate'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [4]:
X_train

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,...,Hometown_Washington,Decision_skill_possess_Analytical,Decision_skill_possess_Behavioral,Decision_skill_possess_Conceptual,Decision_skill_possess_Directive,Compensation_and_Benefits_type0,Compensation_and_Benefits_type1,Compensation_and_Benefits_type2,Compensation_and_Benefits_type3,Compensation_and_Benefits_type4
Employee_ID,,,,,,,,,,,,,,,,,,,,,
EID_10814,34.0,3,5.0,4,40,1,3,4.0,4.0,3,...,0,1,0,0,0,0,0,0,1,0
EID_4658,57.0,3,25.0,3,20,0,1,2.0,2.0,5,...,0,0,1,0,0,0,0,1,0,0
EID_17936,20.0,3,1.0,1,34,0,3,8.0,3.0,3,...,0,0,1,0,0,0,0,1,0,0
EID_24477,26.0,1,4.0,1,42,1,3,2.0,4.0,4,...,0,0,0,1,0,0,0,1,0,0
EID_12200,39.0,3,11.0,2,66,1,2,8.0,2.0,4,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EID_23071,26.0,4,2.0,2,68,2,2,6.0,4.0,3,...,0,0,0,0,1,0,0,0,1,0
EID_15023,NaN,3,NaN,2,29,0,1,6.0,4.0,3,...,0,0,0,1,0,0,0,0,1,0
EID_3851,49.0,3,10.0,3,68,1,1,4.0,2.0,3,...,0,1,0,0,0,0,0,0,1,0


In [5]:
from xgboost import XGBRegressor


# # Define the model
my_model_1 = XGBRegressor(random_state=0)

# Fit the model
my_model_1.fit(X_train, y_train)




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [57]:
from sklearn.metrics import mean_absolute_error

# Get predictions
predictions_1 = my_model_1.predict(X_valid)# Your code here
mae_1 = mean_absolute_error(predictions_1, y_valid)
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 0.08040804819864557


In [53]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit the model
my_model_2.fit(X_train, y_train)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 0.0792402494753486


In [66]:
# Define the model
my_model_3 = XGBRegressor(n_estimators=106, learning_rate=0.05)

# Fit the model
my_model_3.fit(X_train, y_train)

# Get predictions
predictions_3 = my_model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)
print("Mean Absolute Error:" , mae_3)

Mean Absolute Error: 0.07559660140500456


In [67]:
preds_test = my_model_1.predict(X_test)
output = pd.DataFrame({'Employee_ID': X_test.index, 'Attrition_rate': preds_test})
output.to_csv('my_submission.csv', index=False)

In [ ]:
import seaborn as sns
sns.pairplot()

In [33]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)
predictions_4 = model.predict(x_test)
mae_4 = mean_absolute_error(predictions_4, y_valid)
print("Mean Absolute Error:" , mae_3)

SyntaxError: invalid character in identifier (<ipython-input-33-4c6ee9a282d2>, line 4)